# Netflix Recommendation Algorithm

<img src="https://i.pcmag.com/imagery/reviews/05cItXL96l4LE9n02WfDR0h-5.fit_lim.size_1200x630.v1582751026.png" width="40%">

Do not start the exercise until you fully understand the submission guidelines, which can be found here.

For any material-related-questions, ask Ami. For any organization-related-questions, ask Lior.


#### **Read the following instructions carefully:**
1. Write your functions in this notebook only. Do not create Python modules and import them.
Feel free to add code blocks if you need.
2. Answers to qualitative questions should be written in markdown cells (with  LATEX  support). Answers that will be written in commented code blocks will not be checked.
3. Kind reminder: the total of all exercises weight is 50% of the course's grade!

#### **This exercise summarizes the following subjects:**
1. **Netflix's recommendation algorithm:**

  1.1. Content Filtering

  1.2. Collaborative Filtering

  1.3. Filling missing values in the Rating Matrix

  1.4. Matrix Factorization

  1.5. Data correlation finding approaches: SVD, PCA


2. **Applications of SVD and PCA:**

  2.1. Face images compression and recognition using PCA

  2.2. Hand written digits recognition using SVD/PCA

  2.3. Correlations detection of mutations in the genom and different populations using SVD/PCA


# Part 1: Netflix's Recommendation Algorithm

We demonstrated in class how to find similar movies (and recommend movies to a user, similar to a movie he likes), using **Ratings Matrix** where a list of users partially rated a list of movies. In class, we demonstarted 2 ways to handle the Rating Matrix using Excel sheets:

1. Calculate similarities (distances) between movies using the 1 to 5 rating values.
2. Turn the Rating Matrix into a **boolean** matrix where R(i,j)=TRUE if User i rated movie j above some threshold TH (1<=TH<=5).
Note that if TH=1 we get a rating matrix where **any** rating of a movie become TRUE in the boolean rating matrix. As mentioned in class, even such elementary representation is, supresingly, good data to work with.

In [1]:
# All imports
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.metrics import mean_squared_error

import zipfile
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (14.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Add more imports if necessary:

## Question 1 (20 points)

In this question you will write a code that, based on the students' rating matrix (attached to the homework assignment file: our_course_ratings.csv), generates recommendations Ratings Matrix as demonstrated in class.

1.1 Write a function `read_matrix()` that reads `our_course_ratings.csv` file into memory. Do not forget to ignore null values.
  * `our_course_ratings.csv` file is a matrix where each cell contains the rating that student `j` rated movie `i`.

1.2 Write a function `create_boolean_matrix()` that converts our rating matrix into a 1/0 matrix.

1.3 Write a function `recommendation_alg()`:
* The function should prompt the user to enter the first letter of a movie they like.
* Validate the user input to ensure it is a single letter.
* Display a list of movies whose names start with the entered letter.
* Ask the user to choose a movie from the list.
* Calculate movie recommendations based on the chosen movie.
* Print and return a list of k recommended movie names.

1.4 Test your results and run `recommendation_alg()` 3 times:
* Choose the movie **Harry Potter and the Deathly Hallows, Part 2 (2011)**.
* Choose the movie **Star Wars: Episode IV - A New Hope (1977)**.
* Choose your **favorite movies** (one for each student).

In [2]:
def read_matrix(matrix):
  """
  Reads a rating matrix from a CSV file, performs preprocessing, and returns the matrix along with movie names and student names.

  Params:
  - matrix: The CSV file containing the rating matrix.

  Return:
  - A tuple (M, moviesNames, studentsNames) where M is the rating matrix, moviesNames is a series of movie names, and studentsNames is a list of student names.
  """
  lines = []
  with open(matrix, newline='') as f:
    spamreader = csv.reader(f)
    for row in spamreader:
        lines.append(row)

  students = lines[0][2:]
  movies = []
  rating_matrix = []

  for ratings in lines[3:]:
    movies.append(ratings[1])
    rating_matrix.append([int(rating) if rating.isnumeric() else None for rating in ratings[2:]])

  return rating_matrix, movies, students

M, moviesNames, studentsNames = read_matrix('our_course_ratings.csv')

In [3]:
def create_boolean_matrix(matrix, TH = 3):
  """
  Creates a boolean matrix from the given input matrix by setting values below the threshold (TH) to 0 and others to 1.

  Params:
  - matrix: The input matrix to be converted into a boolean matrix.
  - TH: The threshold value; values less than TH are set to 0, and others to 1.

  Returns:
  - The boolean matrix with the same shape as the input matrix.
  """
  return [[
      0 if rating is None or rating < TH else 1
      for rating in movie_ratings
  ] for movie_ratings in matrix]

bool_M = create_boolean_matrix(M)

In [4]:
def recommendation_alg(M, moviesNames, TH=3, k: int = 3):
    """
    Receives a movie the user likes and returns k other movies he might like.

    Params:
    - M: a ratings matrix
    - moviesNames: a movie names matrix
    - TH: the threshold value
    - k: number of best results to return

    Returns:
    - k other movies he might like, by names, not ids
    """
    selected_movie = get_movie_name(moviesNames)
    selected_movie_index = next(index for index, movie in enumerate(moviesNames) if movie == selected_movie)

    binary_rating_matrix = create_boolean_matrix(M, TH)
    selected_movie_ratings = binary_rating_matrix[selected_movie_index]

    movies_by_similarity = sorted(
        (mean_square_error(selected_movie_ratings, other_movie_ratings), movie_index)
        for movie_index, other_movie_ratings in enumerate(binary_rating_matrix)
        if not movie_index == selected_movie_index
    )

    return [
        moviesNames[index] for (_d, index) in movies_by_similarity[:k]
    ]


def get_movie_name(movieNames):
  first_letter_of_movie = None

  while not first_letter_of_movie:
    print("\n\nEnter the first letter of the movie:")
    user_input = input()

    if len(user_input) == 1:
      first_letter_of_movie = user_input

  options = [movie for movie in movieNames if movie.lower().startswith(first_letter_of_movie.lower())]
  options_string = '\n'.join(options)

  movie_selected = None

  while not movie_selected:
    print(f"\n\nPlease select one of:\n{options_string}")
    user_input = input()

    if user_input in options:
      movie_selected = user_input

  print(f"\n\nYou have selected: '{movie_selected}'")

  return movie_selected


def mean_square_error(movie1_ratings, movie2_ratings):
  return sum((rating1 - rating2) ** 2 for (rating1, rating2) in zip(movie1_ratings, movie2_ratings)) / len(movie1_ratings)

# Test the content filtering recommendation algorithm
suggestions = recommendation_alg(M, moviesNames, k=3)
print(f"Suggestions = {suggestions}")

suggestions = recommendation_alg(M, moviesNames, k=3)
print(f"Suggestions = {suggestions}")

suggestions = recommendation_alg(M, moviesNames, k=3)
print(f"Suggestions = {suggestions}")



Enter the first letter of the movie:




Please select one of:
Harry Potter and the Deathly Hallows, Part 2 (2011)
Harry Potter and the Sorcerer's Stone (2001)
Harry Potter and the Deathly Hallows, Part 1 (2010)
Harry Potter and the Order of the Phoenix (2007)
Harry Potter and the Half-Blood Prince (2009)
Harry Potter and the Goblet of Fire (2005)
Harry Potter and the Chamber of Secrets (2002)
Hi, Mom (2021)
Harry Potter and the Prisoner of Azkaban (2004)
Harakiri (1962)
High and Low (1963)
Hamilton (2020)
Heat (1995)
Howl's Moving Castle (2004)
Harry Potter and the Deathly Hallows: Part 2 (2011)
Hacksaw Ridge (2016)
How to Train Your Dragon (2010)
Hotel Rwanda (2004)
Hachi: A Dog's Tale (2009)


Please select one of:
Harry Potter and the Deathly Hallows, Part 2 (2011)
Harry Potter and the Sorcerer's Stone (2001)
Harry Potter and the Deathly Hallows, Part 1 (2010)
Harry Potter and the Order of the Phoenix (2007)
Harry Potter and the Half-Blood Prince (2009)
Harry Potter and the Goblet of Fire (2005)
Harry Potter and the Cha

## Question 2 (40 points)

In this question you will write some functions that, using SVD analysis of the rating matrix, does the following 3 tasks:
1. Show X=3 movies most similar to a certain movie.
1. Shows X=3 users most similar to a certain user.
1. Recommend X=3 movies to a user based on the vector representing the user preferences in martix $V^T$ and the movies data in matrix $U$.


You can look at the following implementations of SVD analysis to make sure your write the SVD analysis correct ([1](https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/)) , ([2](https://www.section.io/engineering-education/singular-value-decomposition-in-python/)).

##### **2.1 Write a function `calculateSpecificRating()` that performs matrix factorization (SVD) to calculate the specific rating of a user for a movie (5 points).**

Follow the steps below:

   * Ensure correct usage of the original rating matrix, $M$.
   
   $M$ should have **users** as its **columns** and **movies** as its **rows**.

   * You are allowed to use built-in Python functions to perform SVD on the ratings matrix $M$.

   * Calculate column i=1 of $M$, using the obtained matrices $U, S, V$ and compare the results to the original $M$.

   * Calculate row j=15 of $M$ using the matrices $U, S, V$ and compare the results to the original $M$.

   Note: If the printed values are **not exactly** the same as the original rating matrix column and row, do **not** proceed. Check your code and fix any discrepancies.

In [5]:
# print(type(M))
# print(type(M[0]))
# print(np.shape(M))
# print(np.ndim(M))

def convert_to_numpy_array(M):
    """
    Converts a matrix to a numpy array.

    Params:
    - M: The matrix to be converted.

    Returns:
    - The converted numpy array.
    """
    max_length = max(len(row) for row in M)
    nested_list = [row + [None] * (max_length - len(row)) for row in M]

    nested_array = np.array(nested_list, dtype=object)
    nested_array= nested_array.astype('float')
    nested_array=np.nan_to_num(nested_array, nan=0)
    nested_array= nested_array.astype('uint8')
    return nested_array

M_numpy=convert_to_numpy_array(M)

# print(type(M_numpy))
# print(M_numpy[41, 0])
# print(np.shape(M_numpy))
# print(np.ndim(M_numpy))


In [6]:
U, S, V = np.linalg.svd(M_numpy, full_matrices=True)

def calculateSpecificRating(U, S, V, row, column):
    """
    Calculate the specific rating of a user for a movie using SVD.

    Params:
    - U: Left singular vector matrix
    - S: Singular values matrix
    - V: Right singular vector matrix
    - row: Index of the movie
    - column: Index of the user

    Returns:
    - The calculated rating for the user at the specified column to the movie at the specified row.
    """
    return calculateMatrix(U, S, V)[row][column]

def calculateSpecificRatingFromMatrix(M, row, column):
    """
    Calculate the specific rating of a user for a movie using SVD.

    Params:
    - U: Left singular vector matrix
    - S: Singular values matrix
    - V: Right singular vector matrix
    - row: Index of the movie
    - column: Index of the user

    Returns:
    - The calculated rating for the user at the specified column to the movie at the specified row.
    """
    return M[row][column]


def calculateMatrix(U, S, V):
    """
    Calculate the specific rating of a user for a movie using SVD.

    Params:
    - U: Left singular vector matrix
    - S: Singular values matrix
    - V: Right singular vector matrix
    - row: Index of the movie
    - column: Index of the user

    Returns:
    - The calculated rating for the user at the specified column to the movie at the specified row.
    """

    smat = np.zeros((U.shape[0], V.shape[0]))
    smat[:V.shape[0], :V.shape[0]] = np.diag(S)
    SV = np.dot(smat,V)
    return np.round(np.dot(U, SV)).astype('uint8')

calculateSpecificRating(U, S, V, 2, 3)
# calculateMatrix(U, S, V)


5

In [7]:
def compareArr(arr1, arr2):
    return np.allclose(arr1, arr2)

In [8]:
col1 = []
col1=calculateMatrix(U, S, V)[:,1]
print("comparison to col1: ", compareArr(col1, M_numpy[:,1]))

row15 = []
row15=calculateMatrix(U, S, V)[15,:]
print("comparison to row15: ", compareArr(row15, M_numpy[15,:]))

comparison to col1:  True
comparison to row15:  True


In [9]:
# These should print the exact same values:
print(f'The value at M[0,1] is: {M_numpy[0,1]}, and the value of the function at (0,1) is: {calculateSpecificRating(U, S, V, 0, 1)}')
print(f'The value at M[0,0] is: {M_numpy[0,0]}, and the value of the function at (0,0) is: {calculateSpecificRating(U, S, V, 0, 0)}')
print(f'The value at M[1,8] is: {M_numpy[1,8]}, and the value of the function at (1,8) is: {calculateSpecificRating(U, S, V, 1, 8)}')

The value at M[0,1] is: 3, and the value of the function at (0,1) is: 3
The value at M[0,0] is: 4, and the value of the function at (0,0) is: 4
The value at M[1,8] is: 5, and the value of the function at (1,8) is: 5


##### **2.2 Implementing SVD Analysis (25 points)**

* Write a function `cutSVD()` that receives matrices $U, S, V$ and an integer `k` and returns the cut matrices $U_{cut}, S_{cut}, V_{cut}$. Ensure that these matrices are cut to the desired size as explained in the class presentation.

* Write a function `dist()` to calculate the similarity of two vectors `x` and `y`. Choose either `Mean Squared Error (MSE)` or `L1 norm` as the similarity metric.

* Write a function `similarMovies()` that receives cut matrices $U_{cut}, S_{cut}, V_{cut}$, a `movie ID`, and an integer `x`. It returns a list of `x`movies most similar to the specified movie based on the SVD analysis.

* Write a function `similarUsers()` that receives cut matrices $U_{cut}, S_{cut}, V_{cut}$, a `user ID`, and an integer `x`. It returns a list of `x` users most similar to the specified user based on the SVD analysis.

* Write a function `SVDFullRecomendationAlgo()` that receives cut matrices $U_{cut}, S_{cut}, V_{cut}$, a `user ID`, and an integer `x`. It returns a list of `x` movies most recommended to the specified user based on all users' preferences in the cut matrices.

In [10]:
def cutSVD(U, S, V, k):
    """
    Cuts the SVD matrices U, S, and V to keep the first k columns of U,
    the first k singular values of S, and the first k rows of V.

    Params:
    - U: U part of SVD
    - S: S part of SVD
    - V: V part of SVD
    - k: how many parameters from the original matrices to keep

    Returns:
    - U_cut, S_cut, V_cut: cut matrices to the desired size
    """
    return U[:, :k], S[:k], V[:k, :]


In [11]:
def dist(x, y):
    """
    Calculates the similarity of 2 movies/users using Mean Squared Error (MSE).

    Params:
    - x: movies/users
    - y: movies/users

    Returns:
    - distance/similarity score
    """

    return mean_squared_error(x, y)

In [12]:
def similarMovies(U_cut, S_cut, V_cut, moviesNames, movie_id, x = 3):
    """
    Receives a movie the user likes and returns x movies he might also like.

    Params:
    - U_cut, S_cut, V_cut: the SVD presentation of the ratings matrix M,
      after it was cut using cutSVD(U, S, V, k)
    - movie_id: the id of the movie the user likes
    - x: number of best results to return

    Returns:
    - x other movies he might like, by names, not ids
    """
    recommendations = []
    movie_index = moviesNames.index(movie_id)
    movie_ratings = U_cut[movie_index, :]
    for i in range(len(U_cut[:, 0])):
        if i != movie_index:
            recommendations.append((dist(movie_ratings, U_cut[i, :]), moviesNames[i]))
    recommendations.sort()
    recommendations = recommendations[:x]
    recommendations = [rec[1] for rec in recommendations]

    assert type(recommendations) is list
    assert type(recommendations[0]) is str
    return recommendations

In [13]:
def similarUsers(U_cut, S_cut, V_cut, studentsNames, user_id, x = 3):
    """
    Receives a user_id the user likes and returns x other users most similar to this user.

    Params:
    - U_cut, S_cut, V_cut: the SVD presentation of the ratings matrix M,
      after it was cut using cutSVD(U, S, V, k)
    - user_id: the id of the user analyzed
    - x: number of best results to return

    Returns:
    - x other users most similar to the user, by names, not ids
    """
    recommendations = []

    user_index = studentsNames.index(user_id)
    user_ratings = V_cut[:, user_index]

    for i in range(len(V_cut[0])):
        if i != user_index:
            recommendations.append((dist(user_ratings, V_cut[:, i]), studentsNames[i]))
    recommendations.sort()
    recommendations = recommendations[:x]
    recommendations = [rec[1] for rec in recommendations]

    assert type(recommendations) is list
    assert type(recommendations[0]) is str
    return recommendations


In [14]:
def SVDFullRecomendationAlgo(U_cut, S_cut, V_cut, user_id, moviesNames, x = 3):
    """
    Receives a user_id and returns x movies most recommended to this user
    Based on ALL the users' preferences as represented in the cut-U-S-V matrices.

    Params:
    - U_cut, S_cut, V_cut: the SVD presentation of the ratings matrix M,
      after it was cut using cutSVD(U, S, V, k)
    - user_id: the id of the user analyzed
    - x: number of best results to return

    Returns:
    - x most recommended movies he might like, by names, not ids
    """
    M_approx = np.matmul(np.matmul(U_cut, np.diag(S_cut)), V_cut)
    user_movie_ratings = M_approx[:, user_id]
    recommendations = [
        moviesNames[rec[1]]
        for rec in sorted(
            ((rating, i) for i, rating in enumerate(user_movie_ratings)),
            reverse=True
        )
    ]

    assert type(recommendations) is list
    assert type(recommendations[0]) is str
    return recommendations[:x]

##### **2.3 Implementing recommendation algorithm using k-elements of SVD analysis of the rating matrix (5 points)**

Let's bring everything together by utilizing `cutSVD()` on the matrices $U, S, V$ to retain only the 1st `k`-columns of $U$, the first `k`-scalars of $S$, and the first `k`-rows of $V$.

Set `k = 10` as a parameter in the code.

Use the cut matrices $U_{cut}, S_{cut}, V_{cut}$ to perform the following tasks:

* Display $x=3$ movies most similar to a certain movie. If the results are not as in question #1, try changing the value of $k$ and see how it affects the results. Provide a brief discussion of the results.

* Display $x=3$ users most similar to a certain user (choose yourself as the user and compare to the rest of the class to you). If the results are not as in question #1, try changing the value of $k$ and see how it affects the results. Discuss briefly the observed changes.

* Recommend $x=3$ movies to a user based on the vector representing the user preferences in matrix $V^T$ and the movies' data in matrix $U$.

In [15]:
k = 10

U_cut, S_cut, V_cut = cutSVD(U, S, V, k)
print(U_cut.shape, S_cut.shape, V_cut.shape)

print("Similar movies:")
print(similarMovies(U_cut, S_cut, V_cut, moviesNames, "Harry Potter and the Deathly Hallows, Part 2 (2011)", x = 3))
print("Similar users:")
print(similarUsers(U_cut, S_cut, V_cut, studentsNames, "Dor S", x = 3))
print("Recommendations to user:")
print(SVDFullRecomendationAlgo(U_cut, S_cut, V_cut, np.where(np.array(studentsNames) == 'Dor S'), moviesNames, x = 3))

(334, 10) (10,) (10, 60)
Similar movies:
['Harry Potter and the Half-Blood Prince (2009)', "Harry Potter and the Sorcerer's Stone (2001)", 'Harry Potter and the Deathly Hallows, Part 1 (2010)']
Similar users:
['Adi Z', 'Daniella Z', 'Yuval D']
Recommendations to user:
['The Lion King (2019)', 'Harry Potter and the Deathly Hallows, Part 2 (2011)', 'Titanic (1997)']


##### **2.4 Comparisings and discussion (5 points)**
Submit the same ids as in 1.2 and show the results with the k-cut SVD method.

Discuss the difference bewtween the outputs here and from question 1.

How many values from the SVD decomposition (ie: what minimum k) gives answers as good as, or better, than the results using the code from question 1?

##### **Answer:**
Very low k starting around 5 already gives reasonable results, that are about as good as the code from question 1. The main difference here is that the data we are calculating over is much more compact, making the SVD method more efficient for approximation results.